In [1]:
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import random as rd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import img_to_array
from keras.optimizers import RMSprop
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
dataset_path = '/kaggle/input/apparel-dataset'
train_sz = 0.75
train_paths = []
test_paths = []

for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        if np.random.rand() < train_sz:
            train_paths.append(path)
        else:
            test_paths.append(path)

print("Total paths:", len(train_paths) + len(test_paths))
print("Train paths:", len(train_paths))
print("Test paths:", len(test_paths))

Total paths: 16170
Train paths: 12130
Test paths: 4040


In [3]:
images = []
labels = []
dims = (96, 96, 3)

for filename in train_paths:
    img = cv.imread(filename)
    if img is None:
        print("Unable to read file:", filename)
        continue
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (96, 96))
    img = img_to_array(img)
    lbl = filename.split(os.path.sep)[-2].split("_")
    images.append(img)
    labels.append(lbl)

train_images = np.array(images, dtype="float") / 255.0
train_labels = np.array(labels)

images = []
labels = []

for filename in test_paths:
    img = cv.imread(filename)
    if img is None:
        print("Unable to read file:", filename)
        continue
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (96, 96))
    img = img_to_array(img)
    lbl = filename.split(os.path.sep)[-2].split("_")
    images.append(img)
    labels.append(lbl)

test_images = np.array(images, dtype="float") / 255.0
test_labels = np.array(labels)

print("Number of train images loaded:", len(train_images))
print("Number of test images loaded:", len(test_images))
print("Shape of train images array:", train_images.shape)
print("Shape of test images array:", test_images.shape)


Number of train images loaded: 12130
Number of test images loaded: 4040
Shape of train images array: (12130, 96, 96, 3)
Shape of test images array: (4040, 96, 96, 3)


In [4]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

print("Class labels:")
print(mlb.classes_)
print("Example binarized label:")
print(labels[0])

Class labels:
['black' 'blue' 'brown' 'dress' 'green' 'hoodie' 'pants' 'pink' 'red'
 'shirt' 'shoes' 'shorts' 'silver' 'skirt' 'suit' 'white' 'yellow']
Example binarized label:
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]


In [4]:
datagen = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                             horizontal_flip=True, validation_split=0.25,
                             rescale=1./255, fill_mode='nearest')

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(96, 96, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(3, 3),  
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(17, activation='softmax')
])

In [6]:
checkpoint = ModelCheckpoint('best_model.h5', save_best_only = True, monitor = 'val_loss', verbose = 1)

In [7]:
onehot_encoder = OneHotEncoder(sparse=False)
train_labels_encoded = onehot_encoder.fit_transform(train_labels.reshape(-1, 1))

onehot_encoder = OneHotEncoder(sparse=False)
test_labels_encoded = onehot_encoder.fit_transform(test_labels.reshape(-1, 1))

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
label_binarizer = MultiLabelBinarizer()
train_labels_encoded = label_binarizer.fit_transform(train_labels)
test_labels_encoded = label_binarizer.transform(test_labels)

In [9]:
model.compile(optimizer= Adam(learning_rate=1e-3, decay=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epoch = 100
BS = 32

history = model.fit(
    x=train_images,
    y=train_labels_encoded,
    batch_size=BS,
    epochs=epoch,
    verbose=1,
    validation_data=(test_images, test_labels_encoded),
    callbacks=[checkpoint]
)

Epoch 1/100
127/380 [=========>....................] - ETA: 4:57 - loss: 15.1417 - accuracy: 0.3728